## Some attempts at Temporal Convolutions

Network inspired by Schirrmeister - performing temporal convolutions

In [1]:
# Imports and setup

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM
from keras.layers import GRU
import matplotlib.pyplot as plt

from utils.load_data import *
from utils.utilities import *
from models.cnn import CNN
from models.rnn import RNN
from models.temporal_cnn import TEMPORAL_CNN
from models.autoencoder import Autoencoder


%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


### Loading Data and Train/Test/Val Splitting

In [2]:
# Loads data from the EEG dataset and removes all EOG data

person_train_val, X_train_val, y_train_val, person_test, X_test, y_test = load_EEG_data()

print ('Training/Valid data shape: {}'.format(X_train_val.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_val.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_val.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [3]:
X_train, y_train, X_val, y_val = split_train_val(X_train_val, y_train_val, percent_validation=0.1)

print ('Training data shape: {}'.format(X_train.shape))
print ('Training target shape: {}'.format(y_train.shape))
print ('Validation data shape: {}'.format(X_val.shape))
print ('Validation target shape: {}'.format(y_val.shape))

Training data shape: (1904, 22, 1000)
Training target shape: (1904,)
Validation data shape: (211, 22, 1000)
Validation target shape: (211,)


### One Hot Encoding for Output Labels

In [4]:
print (y_val[33])
print (y_val.shape)
print (y_test.shape)

y_train_labels = one_hot_encode(y_train)
y_val_labels = one_hot_encode(y_val)
y_test_labels = one_hot_encode(y_test)

print (y_val_labels[33])
print (y_val_labels.shape)
print (y_test_labels.shape)


771
(211,)
(443,)
[0. 0. 1. 0.]
(211, 4)
(443, 4)


### Reshaping input data for temporal CNN

Reshape from (N, E, T) to (N, E, T, 1) - to let CNN know that we are only considering "one channel"

In [5]:
N,E,T = X_train.shape
X_train_tmp = X_train.reshape((N,E,T,1))

N,E,T = X_val.shape
X_val_tmp = X_val.reshape((N,E,T,1))

N,E,T = X_test.shape
X_test_tmp = X_test.reshape((N,E,T,1))

In [8]:
temp_cnn = TEMPORAL_CNN(X_train_tmp, y_train, X_val_tmp, y_val, X_test_tmp, y_test)
temp_cnn.train(epochs=20)

Train on 1904 samples, validate on 211 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1904/1904 [==============================] - 20s 10ms/sample - loss: 4369.6332 - acc: 0.1991 - val_loss: 4272.4093 - val_acc: 0.3649
Epoch 2/20
1904/1904 [==============================] - 19s 10ms/sample - loss: 4272.9744 - acc: 0.3519 - val_loss: 4272.1448 - val_acc: 0.3602
Epoch 3/20
1904/1904 [==============================] - 18s 10ms/sample - loss: 4272.8583 - acc: 0.3057 - val_loss: 4272.0856 - val_acc: 0.2891
Epoch 4/20
1904/1904 [==============================] - 19s 10ms/sample - loss: 4272.8107 - acc: 0.2694 - val_loss: 4272.0498 - val_acc: 0.2844
Epoch 5/20
1904/1904 [==============================] - 18s 10ms/sample - loss: 4272.7812 - acc: 0.2568 - val_loss: 4272.0273 - val_acc: 0.2701
Epoch 6/20
1904/1904 [==============================] - 19s 10ms/sample - loss: 4272.7607 - acc: 0.2563 - val_loss: 4272.0105 - val_acc: 0.2844
Epoch 7/20
1904/1904 [===================